# Handwriting Model Setup #
This part revolves around defining utility functions for the entire pipeline for handwriting analysis.

## Importing Libraries ##
Import the necessary training data from EMNIST to transfer, the tensorflow framework + auxiliary functionality to train a model, and numpy for basic array manipulation.

In [19]:
"""
    This file defines the model to be used for training, transferring learning, and then running the model 
    on student handwriting results.
"""
# dependencies #
!pip install tensorflow
!pip install keras
!pip install emnist

import numpy as np                                                              # array manipulation for weights
import csv                                                                      # saving and loading weights
import tensorflow as tf                                                         # model deployment
import keras                                                                    # model deployment
from tensorflow.keras.models import Sequential                                  # model initialization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense        # model initialization
from tensorflow.keras.preprocessing.image import ImageDataGenerator             # student handwriting training
import emnist                                                                   # transfer learning data
from sklearn.model_selection import train_test_split                            # student handwriting training


"""
    GLOBAL VARIABLES
"""
special_chars = "#%^&*()_-+={}[]\\<>,.?/"
num_chars = 26 + 26 + 10 + len(special_chars) # upper, lower, digits, special_chars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Data Engineering ##
This part of the code will load MNIST data, engineer it to optimize training, then load our new data and engineer it to fit the specifications of the MNIST data as best as we can.

In [20]:
# def shape_data(x_train, y_train, x_test, y_test):
#     # shape
#     x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
#     x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

#     # split
#     y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_chars)
#     y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_chars)
# 
# 
def load_emnist():
    # load mnist for transfer learning
    print("loading data for transfer learning...")
    x_train, y_train = emnist.extract_training_samples('balanced')
    x_test, y_test = emnist.extract_test_samples('balanced')

        # # process data for optimal character recognition
        # print("processing data for optimal results...")
        # x_train, y_train, x_test, y_test = shape_data(x_train, y_train, x_test, y_test)

    return [x_train, x_test, y_train, y_test]


def load_convex_data():
    # load images
    print("loading images for student handwriting...")
    dataset_dir = './student_handwriting/'
    datagen = ImageDataGenerator(rescale=1.0/255.0)  # normalize pixel values between 0 and 1

    # load the dataset using data generator
    train_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=(28, 28),
        color_mode='grayscale',
        batch_size=32,
        class_mode='sparse',
        shuffle=False
    )

    # load x & y data
    x_data = train_generator[0][0]
    y_data = train_generator[0][1]

    # split data
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=17)

    return [x_train, x_test, y_train, y_test]

## Model Deployment ##
This part of the code will train the model on the MNIST data, save the resulting weights for easy initialization in the future, then transfer that learning to the new data for retraining. Throughout the process, weights will be sequentially saved in order to preserve progress and record evolution (for future optimization).

In [21]:
def model_architecture():
    # define model architecture
    print("defining model architecture...")
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    # compile model
    print("compiling model...")
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # return model
    return model


def train_emnist(model, emnist_data):
    # load data
    x_train = emnist_data[0]
    y_train = emnist_data[1]
    x_test = emnist_data[2]
    y_test = emnist_data[3]

    # train model on MNIST
    print("training on MNIST data...")
    model.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_test, y_test))

    # save weights
    save_weights(model)


def save_weights(model):
    # save weights
    print("saving MNIST weights...")
    weights = model.get_weights()
    with open("mnist_weights.csv", 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for weight in weights:
            writer.writerow(weight.flatten())
        

def load_weights(filepath):
    # load from file
    print("loading weights for transfer learning...")
    with open(filepath, 'r') as csvfile:
        reader = csv.reader(csvfile)
        weights = []
        for row in reader:
            weights.append(row.astype(float))
    
    # return weights
    return weights

    
"""
    Transfer the learning
"""
def train_convex_data(model, data):
    # load data
    x_train = data[0]
    y_train = data[1]
    x_test = data[2]
    x_test = data[3]

    # load weights for transfer learning
    print("transferring learning & retraining...")
    transfer_weights = load_weights("mnist_weights.csv")
    model.set_weights(transfer_weights)

    # train on new data
    print("training on pre-processed student handwriting data...")
    model.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_test, y_test))


# Deploy Handwriting Model #
Here's where the utility defined above is run in the order the pipeline requires. This is essentially the high-level pipeline deployment of the CNN.

## Train EMNIST ##
Here, we train the model on the EMNIST dataset for transfer onto our own convex hull data. The size of EMNIST, the lack of data from our own testing, the similarity in contexts between the datasets, and the necessity for accurate readings all drive the need for transfer learning.

We will essentially train a model on the EMNIST dataset, store the resulting weights in the CNN framework, then load those weights for later training/specialization on our own datasets.

At a certain stage in development, transfer learning will become obsolete in this use-case since student data will far surpass the amount of EMNIST data, enabling us to relinquish this dependency and specialize, perhaps offering performance improvements.

In [22]:
print(" ::: STARTED MODEL TRAINING ::: ")

# load data #
emnist_data = load_emnist()

# train model for transfer #
handwriting_model = model_architecture()
handwriting_model = train_emnist(handwriting_model, emnist_data)

# save weights #
save_weights(handwriting_model)

 ::: STARTED MODEL TRAINING ::: 
loading data for transfer learning...
defining model architecture...
compiling model...
training on MNIST data...


ValueError: ignored

## Train on Student Handwriting ##
Here, we train the model on the convex hull data we have produced.

In [ ]:
"""
    Model Testing :: ChatGPT produced model testing example
"""
example_index = 0
example_image = x_test[example_index]
example_label = y_test[example_index]

# Reshape the image to match the input shape of the model
example_image = example_image.reshape(1, 28, 28, 1)

# Make a prediction
prediction = model.predict(example_image)

# Get the predicted label (the index with the highest probability)
predicted_label = tf.argmax(prediction, axis=1)

print("Example:")
print("True Label:", tf.argmax(example_label))
print("Predicted Label:", predicted_label.numpy()[0])
